# East Africa Time Series 

This notebook converts the zonal stats mean values dataframes from output of land_cover_zonal_stats into time series of the 6 environemntal variables for each admin 1 zone across East Africa for all land covers

In [22]:
import numpy as np
import pandas as pd
import os
import collections
from sklearn.preprocessing import StandardScaler

In [7]:
#East Africa Time Series Zonal Stats output datasets
EA_TS_INTERIM = '/home/rgreen/DroughtEDM/Data/EA_TS_16/Interim/'
EA_TS_FINAL = '/home/rgreen/DroughtEDM/Data/EA_TS_16/Final/'
EAST_AFRICA_INTERIM = '/home/rgreen/DroughtEDM/Data/EA_TS_16/EastAfrica_Interim/'
EAST_AFRICA_FINAL = '/home/rgreen/DroughtEDM/Data/EA_TS_16/EastAfrica_Final/'

EA_TS_INTERIM_ANOM = '/home/rgreen/DroughtEDM/Data/EA_TS_16/Interim_Anomaly/'
EA_TS_FINAL_ANOM = '/home/rgreen/DroughtEDM/Data/EA_TS_16/Final_Anomaly/'

In [63]:
EAST_AFRICA_INTERIM_5 = '/home/rgreen/DroughtEDM/Data/EA_TS_16/EastAfrica_Interim/All_Vars/'
EAST_AFRICA_FINAL_5 = '/home/rgreen/DroughtEDM/Data/EA_TS_16/EastAfrica_Final/All_Vars/'

In [64]:
#create list of dekads as datetime objects
begin = pd.datetime(2002,7,1)
end = pd.datetime(2019,4,30)
dtrange = pd.date_range(begin, end)
days = list(range(0,len(dtrange))) #length of dtrange
daysDF = pd.DataFrame({'Days': days}, index=dtrange)
d = daysDF.index.day - np.clip((daysDF.index.day-1) // 10, 0, 2)*10 - 1 #convert daily SM values to dekadal
dekaddates = daysDF.index.values - np.array(d, dtype="timedelta64[D]")
dekads = daysDF.groupby(dekaddates).mean()
dekads = dekads.index

In [65]:
dekads

DatetimeIndex(['2002-07-01', '2002-07-11', '2002-07-21', '2002-08-01',
               '2002-08-11', '2002-08-21', '2002-09-01', '2002-09-11',
               '2002-09-21', '2002-10-01',
               ...
               '2019-01-21', '2019-02-01', '2019-02-11', '2019-02-21',
               '2019-03-01', '2019-03-11', '2019-03-21', '2019-04-01',
               '2019-04-11', '2019-04-21'],
              dtype='datetime64[ns]', length=606, freq=None)

In [68]:
def transform_ts(in_dir, out_dir):
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.csv')
    files = files[tifs]

    
    for filename in files: 
        
        orig_ts = pd.read_csv(in_dir+filename)
        data_list = orig_ts.values.tolist()
        ts_data = [item[6:] for item in data_list] 
        admin_zones = [item[3] for item in data_list]
        
        
        for i in orig_ts.index:
            var_chunk = np.array_split(ts_data[i], 5)
            
#             scaler = StandardScaler()
#             scaled_ndvi = scaler.fit_transform(var_chunk[0])
#             scaled_precip = scaler.fit_transform(var_chunk[1])
#             scaled_lst = scaler.fit_transform(var_chunk[2])
            
            out_df = pd.DataFrame({'SM': var_chunk[0].tolist(),
                                   'NDVI':var_chunk[1].tolist(),
                                   'refET':var_chunk[2].tolist(),
                   'P':var_chunk[3].tolist(),
                   'LST':var_chunk[4].tolist() 
                                   
                  }, index=pd.to_datetime(dekads))
            out_df.index.name = 'Time'
#             features = out_df[['SM','NDVI', 'refET', 'P', 'LST']]
#             scaler = StandardScaler().fit(features.values)

#             features = scaler.transform(features.values)
#             scaled_features = pd.DataFrame(features, columns = ['SM', 'NDVI', 'refET', 'P', 'LST'], index = pd.to_datetime(dekads))
#             scaled_features.index.name = 'Time'
#             print(scaled_features)
            #scaled_features.to_csv(out_dir+str(admin_zones[i])+ '_scaled_' + filename)
            out_df.to_csv(out_dir + 'eastafrica_' + filename)#+str(admin_zones[i])+ '_scaled_' + filename)
            #os.path.splitext(os.path.basename(filename))[0]
            

In [69]:
transform_ts(EAST_AFRICA_INTERIM_5, EAST_AFRICA_FINAL_5)